In [ ]:
using System;
using System.Collections.Generic;
using System.IO;
using System.Linq;
using System.Text.RegularExpressions;

var dataF = File.ReadAllText("inputs/input_day18.txt");

In [ ]:
// Test data
var data = 
//@"[[[[[9,8],1],2],3],4]";

"[7,[6,[5,[4,[3,2]]]]]";

In [ ]:
Pair Parse(string snailfishNumber)
{
    var stack = new Stack<Pair>();

    foreach(var ch in snailfishNumber)
    {
        if(ch == '[')
        {
            stack.Push(new Pair());
            continue;
        }

        if(char.IsDigit(ch))
        {
            stack.Push(new Pair((int)char.GetNumericValue(ch)));
            continue;
        }

        if(ch == ']')
        {
            var right = stack.Pop();
            var left = stack.Pop();
            var pair = stack.Peek();
            pair.SetLeftRight(left, right);
        }
    }

    return stack.Pop();
}


In [ ]:
 public class Pair
 {
    private int _regular;
    //private Pair _pair;

    private bool _isRegular;

    public bool IsRegular => _isRegular || (Left._isRegular && Right._isRegular);

    public int Depth {get; set;}

    private Pair Parent {get; set;}

    public Pair Left {get; set;}
    public Pair Right {get; set;}

    public Pair(Pair left, Pair right)
    {
        Left = left;
        Right = right;

        Left.Parent = this;
        Right.Parent = this;

        IncrementDepth();
    }

    public Pair()
    {}

    public Pair(int value)
    {
        _regular = value;
        _isRegular = true;
    }

    public void SetLeftRight(Pair left, Pair right)
    {
        Left = left;
        Right = right;

        Left.Parent = this;
        Right.Parent = this;

        IncrementDepth();
    }

    public void IncrementValue(int amount)
    {
        if(!_isRegular)
        {
            return;
        }

        _regular += amount;
    }

    public void IncrementDepth()
    {
        if(_isRegular)
        {
            return;
        }

        Depth += 1;

        Left.IncrementDepth();
        Right.IncrementDepth();
    }

    private void ReplaceBy0()
    {
        Left = null;
        Right = null;
        _regular = 0;
        _isRegular = true;
    }

    private (int, Pair) Recurse(Pair pair)
    {
        return pair.IsRegular
            ? (pair.Depth, pair)
            : new[]{Recurse(pair.Left), Recurse(pair.Right)}.MaxBy(_ => _.Item1);
    }

    public Pair FindLeftNeighbour()
    {
        var current = this;

        while (current != null)
        {
            if(current == current.Parent?.Right)
            {
                return current.Parent.Left.FindRegularChild(right: true);
            }

            current = current.Parent;
        }

        return null;
    }

    public Pair FindRightNeighbour()
    {
        var current = this;
        while (current != null)
        {
            if(current == current.Parent?.Left)
            {
                return current.Parent.Right.FindRegularChild(right: false);
            }

            current = current.Parent;
        }

        return null;
    }

    public Pair FindRegularChild(bool right = false)
    {
        var current = this;
        
        while(true)
        {
            if(current._isRegular)
            {
                return current;
            }

            current = 
                right
                    ? current.Right
                    : current.Left;
        }
    }

    public bool DoExplode()
    {
        var (depth, pair) = Recurse(this);

        if(depth == 5)
        {
            var leftNeighbour = pair.FindLeftNeighbour();
            var rightNeighbour = pair.FindRightNeighbour();

            if(leftNeighbour != null)
            {
                leftNeighbour.IncrementValue(pair.Left._regular);
            }
            
            if(rightNeighbour != null)
            {
                rightNeighbour.IncrementValue(pair.Right._regular);
            }

            pair.ReplaceBy0();

            return true;
        }

        return false;
    }

    public void ApplyActions()
    {
        while(DoExplode()){};
    }

    public override string ToString() =>
        _isRegular
            ? _regular.ToString()
            : $"[{Left.ToString()},{Right.ToString()}]";
            //: $"[({Depth}){Left.ToString()},{Right.ToString()}]";
 }

In [ ]:
var pair = Parse("[[[[[9,8],1],2],3],4]");
Console.WriteLine(pair);
pair.DoExplode();
Console.WriteLine(pair);
Console.WriteLine();

pair = Parse("[7,[6,[5,[4,[3,2]]]]]");
Console.WriteLine(pair);
pair.DoExplode();
Console.WriteLine(pair);
Console.WriteLine();

pair = Parse("[[6,[5,[4,[3,2]]]],1]");
Console.WriteLine(pair);
pair.DoExplode();
Console.WriteLine(pair);
Console.WriteLine();

pair = Parse("[[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]");
Console.WriteLine(pair);
//pair.DoExplode();
pair.ApplyActions();
Console.WriteLine(pair);
Console.WriteLine();

pair = Parse("[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]");
Console.WriteLine(pair);
pair.DoExplode();
Console.WriteLine(pair);


[[[[[9,8],1],2],3],4]
[[[[0,9],2],3],4]

[7,[6,[5,[4,[3,2]]]]]
[7,[6,[5,[7,0]]]]

[[6,[5,[4,[3,2]]]],1]
[[6,[5,[7,0]]],3]

[[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]
[[3,[2,[8,0]]],[9,[5,[7,0]]]]

[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]
[[3,[2,[8,0]]],[9,[5,[7,0]]]]
